In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd
books = pd.read_csv('books_cleaned.csv')

In [4]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [15]:
# Export tagged descriptions to a plain text file (one description per line)
books["tagged_description"].to_csv(
    "tagged_description.txt",
    sep=" ",
    index=False,
    header=False,
)


In [20]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1,
    chunk_overlap=0,
    separators="\n"
)
raw_documents = TextLoader(
    ".\\tagged_description.txt",
    encoding="utf-8"
).load()

documents = text_splitter.split_documents(raw_documents)


In [ ]:
import chromadb

print("🚀 ChromaDB başlatılıyor...")

# ChromaDB client
chroma_client = chromadb.Client()

# Eski collection'ı sil
try:
    chroma_client.delete_collection(name="db_books")
    print("✅ Eski collection silindi")
except:
    print("✅ Yeni collection oluşturuluyor")

# Collection oluştur (default embedding - ekstra kurulum gerektirmez)
collection = chroma_client.create_collection(
    name="db_books"
)

print("✅ Collection hazır\n")

# Dokümanları hazırla
texts = [doc.page_content for doc in documents if len(doc.page_content.strip()) > 10]
metadatas = [doc.metadata if doc.metadata else {"source": "unknown"} for doc in documents if len(doc.page_content.strip()) > 10]
ids = [f"doc_{i}" for i in range(len(texts))]

print(f"📚 Toplam doküman: {len(texts)}")
print("⏳ Ekleme başlıyor...\n")

# Batch halinde ekle
batch_size = 100
total = len(texts)
success_count = 0

for i in range(0, total, batch_size):
    end_idx = min(i + batch_size, total)
    
    try:
        collection.add(
            documents=texts[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )
        success_count += (end_idx - i)
        percentage = (end_idx / total) * 100
        print(f"✅ {end_idx}/{total} ({percentage:.1f}%)")
        
    except Exception as e:
        print(f"❌ Batch {i}-{end_idx} hata: {str(e)[:100]}")

print(f"\n🎉 Tamamlandı!")
print(f"📊 Collection'da {collection.count()} doküman var")
print(f"✅ Başarılı: {success_count}/{total}\n")

# Test sorgusu
print("🔍 Test sorgusu yapılıyor...")
try:
    results = collection.query(
        query_texts=["mystery thriller novel"],
        n_results=5
    )
    
    if results['documents'] and results['documents'][0]:
        print("\n📖 En yakın 5 kitap:")
        for i, doc in enumerate(results['documents'][0], 1):
            metadata = results['metadatas'][0][i-1]
            print(f"\n{i}. {doc[:180]}...")
            print(f"   📂 Kaynak: {metadata.get('source', 'N/A')}")
    else:
        print("❌ Sonuç bulunamadı")
        
except Exception as e:
    print(f"❌ Sorgu hatası: {e}")

🚀 ChromaDB başlatılıyor...
✅ Eski collection silindi
✅ Collection hazır

📚 Toplam doküman: 5197
⏳ Ekleme başlıyor...

✅ 100/5197 (1.9%)
✅ 200/5197 (3.8%)
✅ 300/5197 (5.8%)
✅ 400/5197 (7.7%)
✅ 500/5197 (9.6%)
✅ 600/5197 (11.5%)
✅ 700/5197 (13.5%)
✅ 800/5197 (15.4%)
✅ 900/5197 (17.3%)
✅ 1000/5197 (19.2%)
✅ 1100/5197 (21.2%)
✅ 1200/5197 (23.1%)
✅ 1300/5197 (25.0%)
✅ 1400/5197 (26.9%)
✅ 1500/5197 (28.9%)
✅ 1600/5197 (30.8%)
✅ 1700/5197 (32.7%)
✅ 1800/5197 (34.6%)
✅ 1900/5197 (36.6%)
✅ 2000/5197 (38.5%)
✅ 2100/5197 (40.4%)
✅ 2200/5197 (42.3%)
✅ 2300/5197 (44.3%)
✅ 2400/5197 (46.2%)
✅ 2500/5197 (48.1%)
✅ 2600/5197 (50.0%)
✅ 2700/5197 (52.0%)
✅ 2800/5197 (53.9%)
✅ 2900/5197 (55.8%)
✅ 3000/5197 (57.7%)
✅ 3100/5197 (59.6%)
✅ 3200/5197 (61.6%)
✅ 3300/5197 (63.5%)
✅ 3400/5197 (65.4%)
✅ 3500/5197 (67.3%)
✅ 3600/5197 (69.3%)
✅ 3700/5197 (71.2%)
✅ 3800/5197 (73.1%)
✅ 3900/5197 (75.0%)
✅ 4000/5197 (77.0%)
✅ 4100/5197 (78.9%)
✅ 4200/5197 (80.8%)
✅ 4300/5197 (82.7%)
✅ 4400/5197 (84.7%)
✅ 4500/5197 (86.

In [22]:
query = "A book to teach children about nature"

results = collection.query(
    query_texts=[query],
    n_results=5
)

print(f"🔍 Sorgu: '{query}'\n")
print("📊 Sonuçlar (benzerlik skoruna göre):\n")

for i, (doc, distance) in enumerate(zip(results['documents'][0], results.get('distances', [[]])[0]), 1):
    # Distance düşükse benzerlik yüksek demektir
    similarity_score = 1 - distance if distance else "N/A"
    
    print(f"📚 {i}. Kitap (Benzerlik: {similarity_score:.2%})")
    print(f"   {doc[:180]}...")
    print()

🔍 Sorgu: 'A book to teach children about nature'

📊 Sonuçlar (benzerlik skoruna göre):

📚 1. Kitap (Benzerlik: 13.54%)
   
"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of ph...

📚 2. Kitap (Benzerlik: 4.54%)
   
"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to ...

📚 3. Kitap (Benzerlik: 1.30%)
   
"9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a ...

📚 4. Kitap (Benzerlik: 1.05%)
   
"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a...

📚 5. Kitap (Benzerlik: -2.10%)
   
"978

In [25]:
# Kullanıcının ilgi alanına göre kitap bul
query = "A book to teach children about nature"

results = collection.query(
    query_texts=[query],
    n_results=5
)

print(f"🔍 Sorgu: '{query}'\n")
print("📚 Önerilen Kitaplar:\n")

for i, doc in enumerate(results['documents'][0], 1):
    print(f"{i}. {doc[:200]}...")
    print(f"   📂 {results['metadatas'][0][i-1]}\n")

🔍 Sorgu: 'A book to teach children about nature'

📚 Önerilen Kitaplar:

1. 
"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrat...
   📂 {'source': '.\\tagged_description.txt'}

2. 
"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on...
   📂 {'source': '.\\tagged_description.txt'}

3. 
"9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a range of images on a...
   📂 {'source': '.\\tagged_description.txt'}

4. 
"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose li

In [24]:
query = "A book to teach children about nature"

results = collection.query(
    query_texts=[query],
    n_results=5
)

print(f"🔍 Sorgu: '{query}'\n")
print("📊 Sonuçlar (benzerlik skoruna göre):\n")

documents_list = results.get('documents', [[]])[0]
distances_list = results.get('distances', [[]])[0]

for i, doc in enumerate(documents_list, 1):
    # Distance varsa similarity hesapla, yoksa N/A göster
    distance = distances_list[i-1] if distances_list else None
    similarity_score = f"{(1 - distance) * 100:.2f}%" if distance is not None else "N/A"

    print(f"📚 {i}. Kitap (Benzerlik: {similarity_score})")
    print(f"   {doc[:180]}...")  # İlk 180 karakteri göster
    print()


🔍 Sorgu: 'A book to teach children about nature'

📊 Sonuçlar (benzerlik skoruna göre):

📚 1. Kitap (Benzerlik: 13.54%)
   
"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of ph...

📚 2. Kitap (Benzerlik: 4.54%)
   
"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to ...

📚 3. Kitap (Benzerlik: 1.30%)
   
"9780786808397 Introduce your baby to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to exopse little ones to a ...

📚 4. Kitap (Benzerlik: 1.05%)
   
"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a...

📚 5. Kitap (Benzerlik: -2.10%)
   
"978

In [33]:
raw_isbn = results["documents"][0][0].split()[0]
isbn = int(raw_isbn.strip().replace('"', ''))

books[books["isbn13"] == isbn]


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [34]:
books[books["isbn13"]==int(results[0].page_content.split()[0].strip())]

KeyError: 0

In [ ]:
dk_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
HF_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
embedding_model = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    api_key=HF_API_KEY
)


In [50]:
def retrieve_semantic_recommendations(query: str, top_k: int = 10) -> pd.DataFrame:
    # Chroma'da arama
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )

    isbn_list = []

    # Her dökümandaki ISBN'i al
    for doc in results["documents"][0]:
        first_token = doc.split()[0]
        cleaned = first_token.replace('"', "").strip()  # Tırnakları temizle

        # ISBN sayıya dönüştürülebilir mi kontrol et
        try:
            isbn_num = int(cleaned)
            isbn_list.append(isbn_num)
        except:
            continue

    # Eşleşen kitapları döndür
    return books[books["isbn13"].isin(isbn_list)]


In [57]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
1642,9780374522599,0374522596,The Control of Nature,John McPhee,Nature,http://books.google.com/books/content?id=p1qKQ...,The Control of Nature is John McPhee's bestsel...,1990.0,4.24,288.0,3365.0,The Control of Nature,9780374522599 The Control of Nature is John Mc...
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...
3748,9780786808373,0786808373,Baby Einstein: Birds,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=0jxHP...,"Introducing your baby to birds, cats, dogs, an...",2002.0,3.78,20.0,9.0,Baby Einstein: Birds,"9780786808373 Introducing your baby to birds, ..."
3749,9780786808380,0786808381,Baby Einstein: Babies,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=jv4NA...,"Introduce your babies to birds, cats, dogs, an...",2002.0,4.03,20.0,29.0,Baby Einstein: Babies,"9780786808380 Introduce your babies to birds, ..."
3750,9780786808397,078680839X,Baby Einstein: Dogs,Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=qut8t...,"Introduce your baby to birds, cats, dogs, and ...",2002.0,3.81,20.0,26.0,Baby Einstein: Dogs,"9780786808397 Introduce your baby to birds, ca..."
3765,9780786819119,0786819111,"Baby Einstein: Water, Water Everywhere","Disney Book Group,",Juvenile Fiction,http://books.google.com/books/content?id=tuAdA...,Charming illustrations and playful rhythmic ve...,2003.0,3.70,10.0,77.0,"Baby Einstein: Water, Water Everywhere",9780786819119 Charming illustrations and playf...
3834,9780802431486,0802431488,The 10 Commandments of Parenting,H. Edwin Young,Religion,http://books.google.com/books/content?id=Q1Hxj...,Drawn from years of counseling and the author'...,2005.0,4.00,224.0,25.0,The 10 Commandments of Parenting: The Do's and...,9780802431486 Drawn from years of counseling a...


In [60]:
books["categories"].value_counts().reset_index()

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Aged women,1
475,Imperialism,1
476,Human-animal relationships,1
477,Amish,1


In [ ]:
category_mapping = {'fiction':"fiction",
                    'juvenile fiction':"childiren's fiction",
                    'Biography & Autobiography	':"Nonfiction",
                    'history':"Nonfiction",
                    'literary Criticism':"Nonfiction",
                    
                    }